### 최초 한번 실행 과거 데이터 추출용

In [ ]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
import requests
import datetime, time
from datetime import date

import pandas as pd
import numpy as np
import re
import os, sys

In [ ]:
# !pip install chromedriver_autoinstaller

import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

driver = wd.Chrome(driver_name)

In [ ]:
# driver.set_window_position(-10000,0) # hide windows
main_url = 'https://kr.investing.com/'
driver.get(main_url)
time.sleep(1)

In [ ]:
driver.find_element_by_css_selector('#userAccount > div > a.login.bold').click() 
time.sleep(1)

In [ ]:
driver.find_element_by_id('loginForm_password').clear()
driver.find_element_by_id('loginFormUser_email').send_keys('kangeun@naver.com')
time.sleep(1)

In [ ]:
driver.find_element_by_id('loginForm_password').clear()
driver.find_element_by_id('loginForm_password').send_keys(os.environ['investing_pass'])
time.sleep(1)

In [ ]:
driver.find_element_by_css_selector('#signup > a.newButton.orange').click()  
time.sleep(2)

In [ ]:
interest_url = 'https://www.investing.com/economic-calendar/interest-rate-decision-168/'

In [ ]:
driver.get(interest_url)

In [ ]:
for _ in range(3):  # 최근 데이터 3 pages
    driver.find_element_by_id('showMoreHistory168').click()

In [ ]:
df = pd.read_html(driver.page_source)[0] # interest dataframe

In [ ]:
df.columns=  ['release_date', 'time', 'actual_rate', 'forecast_rate', 'prev_rate', 'none']
df['release_date'] = df['release_date'].apply(lambda x : datetime.datetime.strptime(x[:12], "%b %d, %Y"))
df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())

In [ ]:
today = datetime.date.today()
today = today.strftime('%Y%m%d')

In [ ]:
dir_name = 'data/interest_rate/'
start_ix = 'rr'
file_name = 'fed'
i = 1

In [ ]:
filename = '{}{}{}_{}_{}.csv'.format(dir_name, start_ix,i,file_name, today)
df.to_csv(filename, index=False)

In [ ]:
driver.close()
driver.quit()

### Get 한국은행 금리

In [ ]:
kor_url = 'https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643'

In [ ]:
today = datetime.date.today()
today = today.strftime('%Y%m%d')

In [ ]:
df = pd.read_html(kor_url, attrs = {'class': 'fixed'}, flavor=["lxml", "bs4"])[0]

In [ ]:
df.head()

In [ ]:
df.columns=  ['release_yr', 'release_date', 'actual_rate']
df_date_temp = df['release_yr'].astype('str')+df['release_date']
df['date'] = df_date_temp.apply(lambda x : datetime.datetime.strptime(x, "%Y%m월 %d일"))
df = df[['date', 'actual_rate']] # leave only valid columns

In [ ]:
df.head()

In [ ]:
dir_name = 'data/interest_rate/'
start_ix = 'rr'
file_name = 'bok'
i = 2

In [ ]:
filename = '{}{}{}_{}_{}.csv'.format(dir_name, start_ix,i,file_name, today)
df.to_csv(filename, index=False)